In [1]:
from __future__ import absolute_import
from __future__ import print_function
import os
import glob
import random
import numpy as np
from keras import optimizers
from keras.layers import LSTM
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.layers.wrappers import TimeDistributed
from keras.applications.mobilenet import MobileNet
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.layers import Input, InputLayer
from keras.layers.core import Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.applications import imagenet_utils

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
videoFiles = glob.glob('../deep-dataset/numpys/*.npy')
mosFiles = [i for i in videoFiles if 'mos' in i]
videoFiles = [i for i in videoFiles if 'mos' not in i]

In [37]:
def myGenerator():
    while True:
        index_list = random.sample(range(1, 700), 2)
        alldata_x = []
        alldata_y = []
        for i in index_list:
            #print(i, index_list)
            f = videoFiles[i]
            s = f[:-4]+'_mos.npy'
            a = np.load(f)
            b = np.load(s)
            alldata_x.append(a)
            alldata_y.append(b)
        alldata_x = np.array(alldata_x)
        alldata_y = np.array(alldata_y)
        #print(alldata_x.shape, alldata_y.shape)
        yield alldata_x, alldata_y
# x = myGenerator()
# xtrain, ytrain = next(x)
# print('xtrain shape:',xtrain.shape)
# print('ytrain shape:',ytrain.shape)

In [4]:
print(len(videoFiles))
for i in videoFiles:
    print(np.load(i).shape)
    break

803
(200, 68, 120, 3)


In [5]:
height = 68
width = 120
input_shape=(200, height, width, 3)

In [38]:
def mySegNet(input_shape):
    base_model  = MobileNet(input_shape=(224,224,3), include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    cnn_model = Model(inputs=base_model.input, outputs=x)
    
    model = Sequential();
    #model.add(InputLayer(input_shape=input_shape))
    model.add(TimeDistributed(cnn_model, input_shape=input_shape))
    model.add(TimeDistributed(Flatten()))
    #model.add(cnn_model)
    #model.add(Flatten())
    
    model.add(LSTM(512, return_sequences=False))
    model.add(Dense(5, activation='softmax'))
    model.compile(optimizer = optimizers.Adam(lr=0.00001), loss='categorical_crossentropy')
    print(model.summary())
    return model 
#mySegNet(input_shape)

In [ ]:
model = mySegNet(input_shape)
model.fit_generator(generator=myGenerator(),
                    use_multiprocessing=True,
                   steps_per_epoch=20, epochs=50)
model.fit_generator(generator=myGenerator(),
                    use_multiprocessing=True,
                   steps_per_epoch=20, epochs=50)
model.fit_generator(generator=myGenerator(),
                    use_multiprocessing=True,
                   steps_per_epoch=20, epochs=50)
model.save('model1.h5')
model.save_weights('model_weights1.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_13 (TimeDis (None, 200, 1024)         3228864   
_________________________________________________________________
time_distributed_14 (TimeDis (None, 200, 1024)         0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 512)               3147776   
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 2565      
Total params: 6,379,205
Trainable params: 6,357,317
Non-trainable params: 21,888
_________________________________________________________________
None


In [ ]:
def myTestDataGenerator():
    while True:
        index_list = random.sample(range(700, 800), 2)
        alldata_x = []
        alldata_y = []
        for i in index_list:
            #print(i, index_list)
            f = videoFiles[i]
            s = f[:-4]+'_mos.npy'
            a = np.load(f)
            b = np.load(s)
            alldata_x.append(a)
            alldata_y.append(b)
        alldata_x = np.array(alldata_x)
        alldata_y = np.array(alldata_y)
        #print(alldata_x.shape, alldata_y.shape)
        yield alldata_x, alldata_y

In [ ]:
input_shape=(200, height, width, 3)
model = mySegNet(input_shape)
model.load_weights('model_weights1.h5')
predictions = []
ytrue = []
for i in range(0, 100, 2):
    x = myTestDataGenerator()
    xtest, ytest = next(x)
    ytrue.append(ytest)
    pred = model.predict(xtest, batch_size=2)
    for p in pred:
        predictions.append(p)
print('predictions shape: ', np.array(predictions).shape)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
Y = []
for i in ytrue:
    for j in i:
        Y.append(j)
print(np.array(Y).shape)
print(np.array(predictions).shape)
t = []
p = []
for vals in Y:
    for i, val in enumerate(vals):
        if val != 0:
            t.append(i)
for vals in predictions:
    for i, val in enumerate(vals):
        if val == max(vals):
            p.append(i)
print(t, p)
print(accuracy_score(t, p))
print(precision_score(t, p, average='macro'))
print(recall_score(t, p, average='macro'))

In [27]:
x = myGenerator()
xtest, ytest = next(x)
pred = model.predict(xtest)
print(ytest, pred)

[[0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]] [[0.00560889 0.02686344 0.6194726  0.34476787 0.00328722]
 [0.0229902  0.00351662 0.01547285 0.01837347 0.93964684]]
